# Practice of CIFAR10

In [2]:
import tensorflow as tf

print(tf.__version__)

2.2.0-dev20200315


In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, optimizers, Sequential
import os


(x,y), (x_test, y_test) = datasets.cifar10.load_data()
print(x.shape, y.shape, x_test.shape, y_test.shape)

# remove the last division of y and y_test
y = tf.squeeze(y, axis=-1)
y_test = tf.squeeze(y_test, axis=-1)
print(x.shape, y.shape, x_test.shape, y_test.shape)

def preprocess(x,y):
    x = 2*tf.cast(x,dtype=tf.float32)/255.-1
    y = tf.cast(y,dtype=tf.int32)
    return x,y
    
# need to construct Training object with the imported tensors
train_db = tf.data.Dataset.from_tensor_slices((x,y))
# shuffle and preporcess and setup batch
train_db = train_db.shuffle(1000).map(preprocess).batch(128)

test_db = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_db = test_db.map(preprocess).batch(128)

alexnet_layers = [
    # Layer 1
    layers.Conv2D(filters = 96, kernel_size = [11,11], strides = 4, padding = [[0,0],[2,2],[2,2],[0,0]], activation=tf.nn.relu),
    layers.MaxPool2D(pool_size=[3,3], strides=2, padding = 'valid'),
    # Layer 2
    layers.Conv2D(filters = 256, kernel_size = [5,5], strides = 1, padding = [[0,0],[2,2],[2,2],[0,0]], activation=tf.nn.relu),
    layers.MaxPool2D(pool_size = [3,3], strides = 2, padding = 'valid'),
    # Layer 3
    layers.Conv2D(filters = 384, kernel_size = [3,3], strides = 1, padding = [[0,0],[1,1],[1,1],[0,0]], activation=tf.nn.relu),
    # Layer 4
    layers.Conv2D(filters = 384, kernel_size = [3,3], strides = 1, padding = [[0,0],[1,1],[1,1],[0,0]], activation=tf.nn.relu),
    # Layer 5
    layers.Conv2D(filters = 256, kernel_size = [3,3], strides = 1, padding = [[0,0],[1,1],[1,1],[0,0]], activation=tf.nn.relu),
    layers.MaxPool2D(pool_size = [3,3], strides=1, padding = 'same'),
    
    # Before going to FC layers, a batch normal layer is needed.
    layers.BatchNormalization(),
    layers.Flatten(),
    
    # FC layers
    # Layer 6
    layers.Dense(4096, activation = tf.nn.relu),
    layers.Dropout(rate = 0.2),
    # Layer 7
    layers.Dense(4096, activation = tf.nn.relu),
    layers.Dropout(rate = 0.2),
    # Layer 8
    layers.Dense(10, activation=None)
]


def main():
    
    alexnet = Sequential(alexnet_layers)
    alexnet.build(input_shape = [None, 32, 32, 3])
    alexnet.summary()
    #vgg_layers = []
    
    optimizer = optimizers.Adam(lr = 1e-4)
    
    for epoch in range(100):
        for step, (x, y) in enumerate(train_db):
            with tf.GradientTape() as tape:
                out = alexnet(x)
                y_onehot = tf.one_hot(y, depth = 10)

                loss = tf.losses.categorical_crossentropy(y_onehot, out, from_logits = True)
                loss = tf.reduce_mean(loss)

            grads = tape.gradient(loss, alexnet.trainable_variables)
            optimizer.apply_gradients(zip(grads, alexnet.trainable_variables))

            if step % 100 == 0:
                print(epoch, step, 'loss:', float(loss))

        total_num = 0
        total_correct = 0
        for x,y in test_db:
            out = alexnet(x)
            prob = tf.nn.softmax(out, axis = 1)
            pred = tf.argmax(prob, axis = 1)
            pred = tf.cast(pred, dtype = tf.int32)

            correct = tf.cast(tf.equal(pred, y), dtype=tf.int32)
            correct = tf.reduce_sum(correct)

            total_num += x.shape[0]
            total_correct += int(correct)

        acc = total_correct / total_num
        print(epoch, 'acc:', acc)
      
    print("Iteration finished!!")
 
if __name__ == '__main__':
    main()

(50000, 32, 32, 3) (50000, 1) (10000, 32, 32, 3) (10000, 1)
(50000, 32, 32, 3) (50000,) (10000, 32, 32, 3) (10000,)
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              multiple                  34944     
_________________________________________________________________
max_pooling2d (MaxPooling2D) multiple                  0         
_________________________________________________________________
conv2d_1 (Conv2D)            multiple                  614656    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 multiple                  0         
_________________________________________________________________
conv2d_2 (Conv2D)            multiple                  885120    
_________________________________________________________________
conv2d_3 (Conv2D)            multiple                  1327488   
______

40 acc: 0.6212
41 0 loss: 0.0911550298333168
41 100 loss: 0.23329493403434753
41 200 loss: 0.16413424909114838
41 300 loss: 0.09899604320526123
41 acc: 0.6316
42 0 loss: 0.10320775210857391
42 100 loss: 0.18526826798915863
42 200 loss: 0.1241070106625557
42 300 loss: 0.145357146859169
42 acc: 0.6125
43 0 loss: 0.3200679123401642
43 100 loss: 0.1867547333240509
43 200 loss: 0.05369972437620163
43 300 loss: 0.1887812614440918
43 acc: 0.6203
44 0 loss: 0.12022589892148972
44 100 loss: 0.108853779733181
44 200 loss: 0.06529562920331955
44 300 loss: 0.11896373331546783
44 acc: 0.6298
45 0 loss: 0.11959882080554962
45 100 loss: 0.08333136886358261
45 200 loss: 0.12136460840702057
45 300 loss: 0.14014284312725067
45 acc: 0.6292
46 0 loss: 0.09555593132972717
46 100 loss: 0.07337336242198944
46 200 loss: 0.08251248300075531
46 300 loss: 0.1023854911327362
46 acc: 0.6348
47 0 loss: 0.046504076570272446
47 100 loss: 0.10535635054111481
47 200 loss: 0.1755157709121704
47 300 loss: 0.0483800694346

97 200 loss: 0.04609982669353485
97 300 loss: 0.015556344762444496
97 acc: 0.6498
98 0 loss: 0.017875108867883682
98 100 loss: 0.05054555460810661
98 200 loss: 0.02623177506029606
98 300 loss: 0.04815909266471863
98 acc: 0.6397
99 0 loss: 0.02392824925482273
99 100 loss: 0.001988174393773079
99 200 loss: 0.09830549359321594
99 300 loss: 0.005301022902131081
99 acc: 0.6386
Iteration finished!!


In [4]:
import nvidia_smi

nvidia_smi.nvmlInit()

handle = nvidia_smi.nvmlDeviceGetHandleByIndex(0)
# card id 0 hardcoded here, there is also a call to get all available card ids, so we could iterate

info = nvidia_smi.nvmlDeviceGetMemoryInfo(handle)

print("Total memory:", info.total)
print("Free memory:", info.free)
print("Used memory:", info.used)

nvidia_smi.nvmlShutdown()

Total memory: 11553341440
Free memory: 11046354944
Used memory: 506986496
